In [7]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="white")
import os
import plotly.express as px
from plotly.subplots import make_subplots
from plotly import graph_objects as go

### Load all Experiments with a 25 ms walltime

In [8]:
cols={0:"time_string",1:"time_taken_lec",2:"time_taken_safety_controller",3:"total_time",4:"moet",5:"mean_et",6:"avg_iterations"}
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0","1.5"]
obstacles = ["no_obstacles","with_obstacles"]
file_str = "{}/benchmark_experiments_{}_track_porto_{}.csv"
dfs = []
for alg in algorithms:
    for speed in speeds:
        for obs_presence in obstacles:
            filepath = file_str.format(obs_presence,alg,speed)
            df = df = pd.read_csv(filepath,header=None).rename(columns=cols).tail(30)
            df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
            df=df.drop(columns=["time_taken_lec","total_time","time_taken_safety_controller"])
            df["sc_usage"] = 1 - df["ml_usage"]
            df["ml_usage"]=df["ml_usage"]*100
            df["sc_usage"]=df["sc_usage"]*100
            df["ml_controller"] = alg.replace("_rtreach","")
            df["speed"] = speed
            df["obstacle_presence"] = obs_presence
            dfs.append(df)

### Utilities 

In [9]:
def replace_e2e(row):
    if(row=="e2e_image"):
        return "IL (Camera)"
    elif(row=="e2e"):
        return "IL (Lidar)"
    else:
        return row.upper()
def replace_obs(row):
    if(row=="no_obstacles"):
        return "No" 
    else: 
        return "Yes"

### Merge experiments into one Dataframe

In [10]:
all_dfs = pd.concat(dfs)
all_dfs["ml_controller"]=all_dfs["ml_controller"].apply(replace_e2e)
all_dfs["obstacle_presence"] = all_dfs["obstacle_presence"].apply(replace_obs)
all_dfs['moet'] = all_dfs['moet'] *1000
all_dfs['mean_et'] = all_dfs['mean_et'] * 1000
all_dfs["runtime"] = 25

In [11]:
all_dfs

,time_string,moet,mean_et,avg_iterations,ml_usage,sc_usage,ml_controller,speed,obstacle_presence,runtime
0,18/03/2021/16:59:36,57.332,33.6117,6.08480,93.859608,6.140392,IL (Camera),0.5,No,25
1,18/03/2021/17:01:06,59.245,32.8785,6.02632,94.346997,5.653003,IL (Camera),0.5,No,25
2,18/03/2021/17:02:36,59.096,32.9764,6.01465,94.531224,5.468776,IL (Camera),0.5,No,25
3,18/03/2021/17:03:52,56.239,34.0797,6.00288,94.529903,5.470097,IL (Camera),0.5,No,25
4,18/03/2021/17:05:23,59.791,33.4813,6.01381,93.688440,6.311560,IL (Camera),0.5,No,25
...,...,...,...,...,...,...,...,...,...,...
25,19/03/2021/11:31:31,60.683,21.3637,12.33970,12.318856,87.681144,ARS,1.5,Yes,25
26,19/03/2021/11:32:59,59.035,18.9998,13.22620,14.078083,85.921917,ARS,1.5,Yes,25
27,19/03/2021/11:34:12,59.171,26.4688,9.21649,16.938413,83.061587,ARS,1.5,Yes,25
28,19/03/2021/11:35:40,86.777,20.4157,12.77990,12.047074,87.952926,ARS,1.5,Yes,25


### Compute Statistics for all Experiments with 25 ms walltime

In [6]:
res = round(all_dfs.groupby(["ml_controller","speed","obstacle_presence"]).agg(["mean","std","max"]),3)
res

moet                  mean_et         \
                                         mean     std      max    mean    std   
ml_controller speed obstacle_presence                                           
ARS           0.5   No                 74.017  12.212   88.486  26.832  1.681   
                    Yes                60.754   1.337   64.203  25.711  1.046   
              1.0   No                 61.324  11.039  119.473  21.613  3.612   
                    Yes                62.462   8.430  106.071  24.385  4.263   
              1.5   No                 59.191   1.433   62.596  20.316  3.459   
                    Yes                61.328   5.247   86.777  22.118  3.398   
IL (Camera)   0.5   No                 56.984   3.915   61.032  33.737  0.781   
                    Yes                60.257   5.279   85.900  34.038  0.758   
              1.0   No                 58.357   2.224   61.808  33.357  0.569   
                    Yes                62.194  13.091  131.052  32.627  1.427   
              1.5   No                 61.068  12.673  127.728  30.475  2.989   
                    Yes                60.693   2.051   65.607  32.213  1.444   
IL (Lidar)    0.5   No                 53.187   5.298   65.778  36.485  1.103   
                    Yes                59.282   1.946   64.450  34.448  0.527   
              1.0   No                 58.008  16.623  142.887  34.645  5.132   
                    Yes                60.133   5.814   86.629  34.232  2.900   
              1.5   No                 58.132   8.560  100.710  36.112  5.836   
                    Yes                58.864   2.531   68.916  32.198  6.791   
SAC           0.5   No                 86.181   5.213   98.485  28.154  1.830   
                    Yes                61.079   4.255   81.097  26.884  0.908   
              1.0   No                 58.195   1.231   61.225  27.323  5.543   
                    Yes                66.432  36.512  259.612  23.538  3.276   
              1.5   No                 61.423   7.931   98.427  25.942  4.222   
                    Yes                60.309   2.216   68.473  26.155  4.242   

                                              avg_iterations                 \
                                          max           mean    std     max   
ml_controller speed obstacle_presence                                         
ARS           0.5   No                 29.257         10.263  0.230  10.526   
                    Yes                28.784         10.011  0.368  10.394   
              1.0   No                 27.417         11.627  1.628  14.186   
                    Yes                32.570         11.385  1.656  13.847   
              1.5   No                 26.689         12.110  1.589  14.068   
                    Yes                28.309         11.832  1.539  13.715   
IL (Camera)   0.5   No                 37.269          6.011  0.015   6.085   
                    Yes                35.924          6.017  0.031   6.165   
              1.0   No                 34.564          6.183  0.167   6.441   
                    Yes                34.538          6.772  0.703   8.396   
              1.5   No                 34.261          7.688  1.457  11.205   
                    Yes                34.365          6.970  0.761   8.328   
IL (Lidar)    0.5   No                 38.719          6.004  0.019   6.105   
                    Yes                35.533          6.016  0.013   6.079   
              1.0   No                 37.109          6.500  2.141  17.789   
                    Yes                37.492          6.979  1.180  13.051   
              1.5   No                 48.308          6.681  2.130  17.913   
                    Yes                36.646          7.269  2.905  17.864   
SAC           0.5   No                 31.243          9.601  0.257  10.107   
                    Yes                29.850          9.508  0.162   9.893   
              1.0   No              

In [12]:
all_dfs.shape

(720, 10)

### Seperate Experiments into those with Obstacles and those without for latex code generation

In [13]:
with_obs = all_dfs[all_dfs.obstacle_presence=="Yes"]
no_obs = all_dfs[all_dfs.obstacle_presence=="No"]

In [14]:
with_obs[list(with_obs.columns[with_obs.columns.isin(['moet','mean_et','avg_iterations','speed'])])].groupby("speed").agg(["mean","std"])

moet               mean_et           avg_iterations          
            mean        std       mean       std           mean       std
speed                                                                    
0.5    60.343092   3.609197  30.270057  4.097035       7.887978  1.898403
1.0    62.805250  19.935807  28.695691  5.710042       8.998741  2.518410
1.5    60.298433   3.364057  28.170907  6.116465       8.941194  2.726182

### Create relevant latex code

In [15]:
with_obs
res = round(with_obs.drop(columns="moet").groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]).sort_values(by=["ml_controller"]),2)
print(res[res.columns[:-4]].to_latex())

\begin{tabular}{lllrrrr}
\toprule
    &     &    & \multicolumn{2}{l}{mean\_et} & \multicolumn{2}{l}{avg\_iterations} \\
    &     &    &    mean &   std &           mean &   std \\
ml\_controller & speed & runtime &         &       &                &       \\
\midrule
ARS & 0.5 & 25 &   25.71 &  1.05 &          10.01 &  0.37 \\
    & 1.0 & 25 &   24.38 &  4.26 &          11.38 &  1.66 \\
    & 1.5 & 25 &   22.12 &  3.40 &          11.83 &  1.54 \\
IL (Camera) & 0.5 & 25 &   34.04 &  0.76 &           6.02 &  0.03 \\
    & 1.0 & 25 &   32.63 &  1.43 &           6.77 &  0.70 \\
    & 1.5 & 25 &   32.21 &  1.44 &           6.97 &  0.76 \\
IL (Lidar) & 0.5 & 25 &   34.45 &  0.53 &           6.02 &  0.01 \\
    & 1.0 & 25 &   34.23 &  2.90 &           6.98 &  1.18 \\
    & 1.5 & 25 &   32.20 &  6.79 &           7.27 &  2.91 \\
SAC & 0.5 & 25 &   26.88 &  0.91 &           9.51 &  0.16 \\
    & 1.0 & 25 &   23.54 &  3.28 &          10.86 &  1.60 \\
    & 1.5 & 25 &   26.16 &  4.24 &          

In [16]:
### With Obstacles

In [17]:
res = round(with_obs.drop(columns="moet").groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]).sort_values(by=["ml_controller"]),2)
res[res.columns[4:]]
print(res[res.columns[4:]].to_latex())

\begin{tabular}{lllrrrr}
\toprule
    &     &    & \multicolumn{2}{l}{ml\_usage} & \multicolumn{2}{l}{sc\_usage} \\
    &     &    &     mean &    std &     mean &    std \\
ml\_controller & speed & runtime &          &        &          &        \\
\midrule
ARS & 0.5 & 25 &    55.26 &   2.95 &    44.74 &   2.95 \\
    & 1.0 & 25 &    26.39 &   3.81 &    73.61 &   3.81 \\
    & 1.5 & 25 &    14.11 &   2.04 &    85.89 &   2.04 \\
IL (Camera) & 0.5 & 25 &    84.75 &   2.68 &    15.25 &   2.68 \\
    & 1.0 & 25 &    49.73 &   8.18 &    50.27 &   8.18 \\
    & 1.5 & 25 &    36.04 &  10.78 &    63.96 &  10.78 \\
IL (Lidar) & 0.5 & 25 &    92.46 &   1.63 &     7.54 &   1.63 \\
    & 1.0 & 25 &    52.30 &   6.35 &    47.70 &   6.35 \\
    & 1.5 & 25 &    25.66 &  10.15 &    74.34 &  10.15 \\
SAC & 0.5 & 25 &    53.29 &   3.66 &    46.71 &   3.66 \\
    & 1.0 & 25 &    18.97 &   3.15 &    81.03 &   3.15 \\
    & 1.5 & 25 &     9.25 &   2.33 &    90.75 &   2.33 \\
\bottomrule
\end{tabular}



### No obstacles

In [18]:
res = round(no_obs.drop(columns="moet").groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]).sort_values(by=["ml_controller"]),2)

res=res.reset_index()
print(res[res.columns[3:7]].to_latex())

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{2}{l}{mean\_et} & \multicolumn{2}{l}{avg\_iterations} \\
{} &    mean &   std &           mean &   std \\
\midrule
0  &   26.83 &  1.68 &          10.26 &  0.23 \\
1  &   21.61 &  3.61 &          11.63 &  1.63 \\
2  &   20.32 &  3.46 &          12.11 &  1.59 \\
3  &   33.74 &  0.78 &           6.01 &  0.02 \\
4  &   33.36 &  0.57 &           6.18 &  0.17 \\
5  &   30.48 &  2.99 &           7.69 &  1.46 \\
6  &   36.49 &  1.10 &           6.00 &  0.02 \\
7  &   34.65 &  5.13 &           6.50 &  2.14 \\
8  &   36.11 &  5.84 &           6.68 &  2.13 \\
9  &   28.15 &  1.83 &           9.60 &  0.26 \\
10 &   27.32 &  5.54 &           9.01 &  2.52 \\
11 &   25.94 &  4.22 &           9.70 &  1.89 \\
\bottomrule
\end{tabular}



###  Experiments 10 ms walltime

In [19]:
cols={0:"time_string",1:"time_taken_lec",2:"time_taken_safety_controller",3:"total_time",4:"moet",5:"mean_et",6:"avg_iterations"}
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0","1.5"]
file_str = "10ms/benchmark_experiments_{}_track_porto_{}.csv"
dfs2 = []
for alg in algorithms:
    for speed in speeds:
            filepath = file_str.format(alg,speed)
            try:
                df = df = pd.read_csv(filepath,header=None).rename(columns=cols).tail(30)
            except:
                print(filepath)
                continue
            df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
            df=df.drop(columns=["time_taken_lec","total_time","time_taken_safety_controller"])
            df["sc_usage"] = 1 - df["ml_usage"]
            df["ml_usage"]=df["ml_usage"]*100
            df["sc_usage"]=df["sc_usage"]*100
            df["ml_controller"] = alg.replace("_rtreach","")
            df["speed"] = speed
            df["obstacle_presence"] = obs_presence
            df["runtime"] = 10
            dfs2.append(df)

In [20]:
all_dfs2 = pd.concat(dfs2)
all_dfs2["ml_controller"]=all_dfs2["ml_controller"].apply(replace_e2e)
all_dfs2["obstacle_presence"] = all_dfs2["obstacle_presence"].apply(replace_obs)
all_dfs2.groupby(["ml_controller","obstacle_presence","speed"]).count()
all_dfs2['moet'] = all_dfs2['moet'] *1000
all_dfs2['mean_et'] = all_dfs2['mean_et'] * 1000
all_dfs2["runtime"] = 10
all_dfs.shape

(720, 10)

In [21]:
res2 = round(all_dfs2.groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]).sort_values(by=["ml_controller"]),2)
res2

moet        mean_et       avg_iterations        \
                              mean    std    mean   std           mean   std   
ml_controller speed runtime                                                    
ARS           0.5   10       24.81   1.35   11.24  0.47           9.13  0.34   
              1.0   10       26.02   6.37    9.92  1.71          10.65  1.80   
              1.5   10       29.09  16.82    9.72  1.54          10.72  1.63   
IL (Camera)   0.5   10       23.23   1.61   16.00  0.39           4.99  0.01   
              1.0   10       28.37  18.91   14.84  0.99           6.09  1.05   
              1.5   10       32.45  14.93   15.06  0.79           6.01  0.72   
IL (Lidar)    0.5   10       25.11   1.74   16.38  0.36           4.99  0.01   
              1.0   10       28.08   6.70   14.70  2.42           6.48  2.44   
              1.5   10       31.19  20.83   12.86  5.09           8.66  5.01   
SAC           0.5   10       24.00   1.74   12.04  0.34           8.56  0.20   
              1.0   10       32.33  39.49   10.68  1.29           9.94  1.42   
              1.5   10       26.15   4.23   10.90  1.58           9.98  1.35   

                            ml_usage        sc_usage         
                                mean    std     mean    std  
ml_controller speed runtime                                  
ARS           0.5   10         55.19   2.43    44.81   2.43  
              1.0   10         26.44   3.52    73.56   3.52  
              1.5   10         14.39   1.89    85.61   1.89  
IL (Camera)   0.5   10         84.89   2.52    15.11   2.52  
              1.0   10         46.96  10.96    53.04  10.96  
              1.5   10         35.65   8.01    64.35   8.01  
IL (Lidar)    0.5   10         92.22   2.20     7.78   2.20  
              1.0   10         51.01  10.55    48.99  10.55  
              1.5   10         23.18  12.23    76.82  12.23  
SAC           0.5   10         53.66   4.35    46.34   4.35  
              1.0   10         17.85   2.24    82.15   2.24  
              1.5   10          9.57   2.36    90.43   2.36

In [22]:
res = round(with_obs.groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]).sort_values(by=["ml_controller"]),2)
res

moet        mean_et       avg_iterations        \
                              mean    std    mean   std           mean   std   
ml_controller speed runtime                                                    
ARS           0.5   25       60.75   1.34   25.71  1.05          10.01  0.37   
              1.0   25       62.46   8.43   24.38  4.26          11.38  1.66   
              1.5   25       61.33   5.25   22.12  3.40          11.83  1.54   
IL (Camera)   0.5   25       60.26   5.28   34.04  0.76           6.02  0.03   
              1.0   25       62.19  13.09   32.63  1.43           6.77  0.70   
              1.5   25       60.69   2.05   32.21  1.44           6.97  0.76   
IL (Lidar)    0.5   25       59.28   1.95   34.45  0.53           6.02  0.01   
              1.0   25       60.13   5.81   34.23  2.90           6.98  1.18   
              1.5   25       58.86   2.53   32.20  6.79           7.27  2.91   
SAC           0.5   25       61.08   4.26   26.88  0.91           9.51  0.16   
              1.0   25       66.43  36.51   23.54  3.28          10.86  1.60   
              1.5   25       60.31   2.22   26.16  4.24           9.69  1.72   

                            ml_usage        sc_usage         
                                mean    std     mean    std  
ml_controller speed runtime                                  
ARS           0.5   25         55.26   2.95    44.74   2.95  
              1.0   25         26.39   3.81    73.61   3.81  
              1.5   25         14.11   2.04    85.89   2.04  
IL (Camera)   0.5   25         84.75   2.68    15.25   2.68  
              1.0   25         49.73   8.18    50.27   8.18  
              1.5   25         36.04  10.78    63.96  10.78  
IL (Lidar)    0.5   25         92.46   1.63     7.54   1.63  
              1.0   25         52.30   6.35    47.70   6.35  
              1.5   25         25.66  10.15    74.34  10.15  
SAC           0.5   25         53.29   3.66    46.71   3.66  
              1.0   25         18.97   3.15    81.03   3.15  
              1.5   25          9.25   2.33    90.75   2.33

In [23]:
varying_experiments = pd.concat([res,res2])
experiment_tables = varying_experiments.sort_values(by=['ml_controller','speed','runtime'])
cols = list(experiment_tables.sort_values(by=['ml_controller','speed','runtime']).columns)
rel_columns = cols[:7] + cols[8:]
print(experiment_tables[rel_columns].to_latex())

\begin{tabular}{lllrrrrrrrrr}
\toprule
    &     &    & \multicolumn{2}{l}{moet} & \multicolumn{2}{l}{mean\_et} & \multicolumn{2}{l}{avg\_iterations} & ml\_usage & \multicolumn{2}{l}{sc\_usage} \\
    &     &    &   mean &    std &    mean &   std &           mean &   std &     mean &     mean &    std \\
ml\_controller & speed & runtime &        &        &         &       &                &       &          &          &        \\
\midrule
ARS & 0.5 & 10 &  24.81 &   1.35 &   11.24 &  0.47 &           9.13 &  0.34 &    55.19 &    44.81 &   2.43 \\
    &     & 25 &  60.75 &   1.34 &   25.71 &  1.05 &          10.01 &  0.37 &    55.26 &    44.74 &   2.95 \\
    & 1.0 & 10 &  26.02 &   6.37 &    9.92 &  1.71 &          10.65 &  1.80 &    26.44 &    73.56 &   3.52 \\
    &     & 25 &  62.46 &   8.43 &   24.38 &  4.26 &          11.38 &  1.66 &    26.39 &    73.61 &   3.81 \\
    & 1.5 & 10 &  29.09 &  16.82 &    9.72 &  1.54 &          10.72 &  1.63 &    14.39 &    85.61 &   1.89 \\
    & 

In [24]:
experiment_tables

moet        mean_et       avg_iterations        \
                              mean    std    mean   std           mean   std   
ml_controller speed runtime                                                    
ARS           0.5   10       24.81   1.35   11.24  0.47           9.13  0.34   
                    25       60.75   1.34   25.71  1.05          10.01  0.37   
              1.0   10       26.02   6.37    9.92  1.71          10.65  1.80   
                    25       62.46   8.43   24.38  4.26          11.38  1.66   
              1.5   10       29.09  16.82    9.72  1.54          10.72  1.63   
                    25       61.33   5.25   22.12  3.40          11.83  1.54   
IL (Camera)   0.5   10       23.23   1.61   16.00  0.39           4.99  0.01   
                    25       60.26   5.28   34.04  0.76           6.02  0.03   
              1.0   10       28.37  18.91   14.84  0.99           6.09  1.05   
                    25       62.19  13.09   32.63  1.43           6.77  0.70   
              1.5   10       32.45  14.93   15.06  0.79           6.01  0.72   
                    25       60.69   2.05   32.21  1.44           6.97  0.76   
IL (Lidar)    0.5   10       25.11   1.74   16.38  0.36           4.99  0.01   
                    25       59.28   1.95   34.45  0.53           6.02  0.01   
              1.0   10       28.08   6.70   14.70  2.42           6.48  2.44   
                    25       60.13   5.81   34.23  2.90           6.98  1.18   
              1.5   10       31.19  20.83   12.86  5.09           8.66  5.01   
                    25       58.86   2.53   32.20  6.79           7.27  2.91   
SAC           0.5   10       24.00   1.74   12.04  0.34           8.56  0.20   
                    25       61.08   4.26   26.88  0.91           9.51  0.16   
              1.0   10       32.33  39.49   10.68  1.29           9.94  1.42   
                    25       66.43  36.51   23.54  3.28          10.86  1.60   
              1.5   10       26.15   4.23   10.90  1.58           9.98  1.35   
                    25       60.31   2.22   26.16  4.24           9.69  1.72   

                            ml_usage        sc_usage         
                                mean    std     mean    std  
ml_controller speed runtime                                  
ARS           0.5   10         55.19   2.43    44.81   2.43  
                    25         55.26   2.95    44.74   2.95  
              1.0   10         26.44   3.52    73.56   3.52  
                    25         26.39   3.81    73.61   3.81  
              1.5   10         14.39   1.89    85.61   1.89  
                    25         14.11   2.04    85.89   2.04  
IL (Camera)   0.5   10         84.89   2.52    15.11   2.52  
                    25         84.75   2.68    15.25   2.68  
              1.0   10         46.96  10.96    53.04  10.96  
                    25         49.73   8.18    50.27   8.18  
              1.5   10         35.65   8.01    64.35   8.01  
                    25         36.04  10.78    63.96  10.78  
IL (Lidar)    0.5   10         92.22   2.20     7.78   2.20  
                    25         92.46   1.63     7.54   1.63  
              1.0   10         51.01  10.55    48.99  10.55  
                    25         52.30   6.35    47.70   6.35  
              1.5   10         23.18  12.23    76.82  12.23  
                    25         25.66  10.15    74.34  10.15  
SAC           0.5   10         53.66   4.35    46.34   4.35  
                    25         53.29   3.66    46.71   3.66  
              1.0   10         17.85   2.24    82.15   2.24  
                    25         18.97   3.15    81.03   3.15  
              1.5   10          9.57   2.36    90.43   2.36  
                    25          9.25   2.33    90.75   2.33

In [25]:
cols = list(experiment_tables.sort_values(by=['ml_controller','speed','runtime']).columns)
rel_columns = cols[:7] + cols[8:]
experiment_tables[rel_columns ]

moet        mean_et       avg_iterations        \
                              mean    std    mean   std           mean   std   
ml_controller speed runtime                                                    
ARS           0.5   10       24.81   1.35   11.24  0.47           9.13  0.34   
                    25       60.75   1.34   25.71  1.05          10.01  0.37   
              1.0   10       26.02   6.37    9.92  1.71          10.65  1.80   
                    25       62.46   8.43   24.38  4.26          11.38  1.66   
              1.5   10       29.09  16.82    9.72  1.54          10.72  1.63   
                    25       61.33   5.25   22.12  3.40          11.83  1.54   
IL (Camera)   0.5   10       23.23   1.61   16.00  0.39           4.99  0.01   
                    25       60.26   5.28   34.04  0.76           6.02  0.03   
              1.0   10       28.37  18.91   14.84  0.99           6.09  1.05   
                    25       62.19  13.09   32.63  1.43           6.77  0.70   
              1.5   10       32.45  14.93   15.06  0.79           6.01  0.72   
                    25       60.69   2.05   32.21  1.44           6.97  0.76   
IL (Lidar)    0.5   10       25.11   1.74   16.38  0.36           4.99  0.01   
                    25       59.28   1.95   34.45  0.53           6.02  0.01   
              1.0   10       28.08   6.70   14.70  2.42           6.48  2.44   
                    25       60.13   5.81   34.23  2.90           6.98  1.18   
              1.5   10       31.19  20.83   12.86  5.09           8.66  5.01   
                    25       58.86   2.53   32.20  6.79           7.27  2.91   
SAC           0.5   10       24.00   1.74   12.04  0.34           8.56  0.20   
                    25       61.08   4.26   26.88  0.91           9.51  0.16   
              1.0   10       32.33  39.49   10.68  1.29           9.94  1.42   
                    25       66.43  36.51   23.54  3.28          10.86  1.60   
              1.5   10       26.15   4.23   10.90  1.58           9.98  1.35   
                    25       60.31   2.22   26.16  4.24           9.69  1.72   

                            ml_usage sc_usage         
                                mean     mean    std  
ml_controller speed runtime                           
ARS           0.5   10         55.19    44.81   2.43  
                    25         55.26    44.74   2.95  
              1.0   10         26.44    73.56   3.52  
                    25         26.39    73.61   3.81  
              1.5   10         14.39    85.61   1.89  
                    25         14.11    85.89   2.04  
IL (Camera)   0.5   10         84.89    15.11   2.52  
                    25         84.75    15.25   2.68  
              1.0   10         46.96    53.04  10.96  
                    25         49.73    50.27   8.18  
              1.5   10         35.65    64.35   8.01  
                    25         36.04    63.96  10.78  
IL (Lidar)    0.5   10         92.22     7.78   2.20  
                    25         92.46     7.54   1.63  
              1.0   10         51.01    48.99  10.55  
                    25         52.30    47.70   6.35  
              1.5   10         23.18    76.82  12.23  
                    25         25.66    74.34  10.15  
SAC           0.5   10         53.66    46.34   4.35  
                    25         53.29    46.71   3.66  
              1.0   10         17.85    82.15   2.24  
                    25         18.97    81.03   3.15  
              1.5   10          9.57    90.43   2.36  
                    25          9.25    90.75   2.33